In [34]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os, random

import tensorflow as tf

from tensorflow.keras import Model, backend as K
from tensorflow.keras.layers import Layer, Input, Dense, Conv2D, LeakyReLU, Flatten, BatchNormalization, AvgPool2D, Reshape, Add, UpSampling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Progbar

In [35]:
# Seed set for reproducibility
random_seed = 2103
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

In [36]:
epochs = 30
batch_size = 32 # 128
dl_size = 64 # 512
rotation_range = 10
real_shape = (64, 64)

In [37]:
def listdir(path, validExts=(".jpg", ".jpeg", ".png", ".bmp"), recursive=False):
    file_list = []
    
    for rootDir, dirnames, filenames in os.walk(path):
        for filename in filenames:
            ext = filename[filename.rfind("."):].lower()
            if ext.endswith(validExts):
                file_list.append(os.path.join(rootDir, filename).replace(" ", "\\ "))
         
        if recursive:
            for dirname in dirnames:
                file_list += listdir(os.path.join(rootDir, dirname).replace(" ", "\\ "), validExts,
                                     recursive=True)
    
    return file_list

file_list = []

directory = 'D:\\Inventions\\Gen3dAnime\\Data\\animefaces-variant2\\faces\\'
file_list += listdir(directory, recursive=True)

random.shuffle(file_list)

files = len(file_list)
print(files)

51223


In [38]:
def MyDisc():
    x = Input(real_shape+(3,))
    y = Dense(1)(x)
    return Model(x, y)

In [52]:
def MyGen():
    z = Input((dl_size,), name="LatentZ")

    y = Dense(real_shape[0]*real_shape[1]*3)(z)
    y = Reshape(real_shape+(3,))(y)
                            
    return Model(z, y)

In [53]:
gen = MyGen()
disc = MyDisc()

gen.summary()
disc.summary()

real = Input(real_shape+(3,))
z = Input((dl_size,))
fake = gen(z)
disc_r = disc(real)
disc_f = disc(fake)

# Generator and discriminator losses according to RaGAN described in Jolicoeur-Martineau (2018).
# Credits to Smith42 on GitHub for the implementation. (https://github.com/Smith42/keras-relativistic-gan)
def relGenLoss(y_true, y_pred):
    epsilon=0.000001
    return -(K.mean(K.log(K.sigmoid(disc_f - K.mean(disc_r, axis=0))+epsilon), axis=0)\
            +K.mean(K.log(1-K.sigmoid(disc_r - K.mean(disc_f, axis=0))+epsilon), axis=0))

def relDiscLoss(y_true, y_pred):
    epsilon=0.000001
    return -(K.mean(K.log(K.sigmoid(disc_r - K.mean(disc_f, axis=0))+epsilon), axis=0)\
            +K.mean(K.log(1-K.sigmoid(disc_f - K.mean(disc_r, axis=0))+epsilon), axis=0))

gen.trainable = True
disc.trainable = False
gen_train = Model([real, z], [disc_r, disc_f])
gen_train.compile(SGD(2e-3, .5, nesterov=True), [relGenLoss, None])
gen_train.summary()

gen.trainable = False
disc.trainable = True
disc_train = Model([real, z], [disc_r, disc_f])
disc_train.compile(SGD(3e-3, .5, nesterov=True), [relDiscLoss, None])
disc_train.summary()


Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LatentZ (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_25 (Dense)             (None, 12288)             798720    
_________________________________________________________________
reshape_7 (Reshape)          (None, 64, 64, 3)         0         
Total params: 798,720
Trainable params: 798,720
Non-trainable params: 0
_________________________________________________________________
Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 64, 64, 3)]       0         
_________________________________________________________________
dense_26 (Dense)             (None, 64, 64, 1)         4         
Total params: 4
Trainable params: 

In [48]:
def dataGenerator():
    image_data_generator = ImageDataGenerator(rotation_range = 10*alpha,
                                              horizontal_flip = True,
                                              preprocessing_function = lambda x: x/127.5-1)
    
    for i in range(files//batch_size):
        images = []
        
        for j in range(batch_size):
            im = cv2.imread(file_list[i*batch_size+j])
            im = cv2.resize(im, real_shape)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            
            images.append(np.array(im))

        yield next(image_data_generator.flow(np.array(images),
                                             batch_size=batch_size,
                                             shuffle=False))

In [55]:
dummy_y = np.zeros((batch_size,))
gen_loss = []
disc_loss = []

nplots = 5
figsize = real_shape

for e in range(epochs):
    target = files//batch_size
    progbar = Progbar(target)
    data_generator = dataGenerator()
    
    print('Epoch ', e+1, '/', epochs)
    for i in range(target):
        progbar.update(i+1)
        
        gen.trainable = False
        disc.trainable = True
        data = next(data_generator)
        z = np.random.normal(scale=.25, size=(batch_size, dl_size))
        z2 = np.random.normal(scale=.25, size=(batch_size, dl_size))
        disc_loss.append(disc_train.train_on_batch([data, z, z2], dummy_y))
        
        gen.trainable = True
        disc.trainable = False
        gen_loss.append(gen_train.train_on_batch([data, z, z2], dummy_y))


Epoch  1 / 30
   1/1600 [..............................] - ETA: 0s

ValueError: in converted code:

    C:\Users\Hudson\Anaconda3\envs\deep-learning-gpu\lib\site-packages\tensorflow_core\python\keras\engine\training_eager.py:305 train_on_batch  *
        outs, total_loss, output_losses, masks = (
    C:\Users\Hudson\Anaconda3\envs\deep-learning-gpu\lib\site-packages\tensorflow_core\python\keras\engine\training_eager.py:273 _process_single_batch
        model.optimizer.apply_gradients(zip(grads, trainable_weights))
    C:\Users\Hudson\Anaconda3\envs\deep-learning-gpu\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py:426 apply_gradients
        grads_and_vars = _filter_grads(grads_and_vars)
    C:\Users\Hudson\Anaconda3\envs\deep-learning-gpu\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py:1039 _filter_grads
        ([v.name for _, v in grads_and_vars],))

    ValueError: No gradients provided for any variable: ['dense_26/kernel:0', 'dense_26/bias:0'].
